# PHE SEIR Model

In this notebook we present how to use the `epimodels` module to set up an instantiation of the model built by Public Health England in collaboration with University of Cambridge, using region specific contact matrices from 2021.

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.stats import gamma
import epimodels as em
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt

### Define setup matrices for the PHE Model

In [2]:
# Populate the model
regions = ['EE', 'London', 'Mid', 'NE', 'NW', 'SE', 'SW']
age_groups = ['0-1', '1-5', '5-15', '15-25', '25-45', '45-65', '65-75', '75+']
weeks = list(range(1,15))

matrices_region = []

# Initial state of the system
for w in weeks:
    weeks_matrices_region = []
    for r in regions:
        path = os.path.join('../data/final_contact_matrices/{}_W{}.csv'.format(r, w))
        region_data_matrix = pd.read_csv(path, header=None, dtype=np.float64)
        regional = em.RegionMatrix(r, age_groups, region_data_matrix)
        weeks_matrices_region.append(regional)

    matrices_region.append(weeks_matrices_region)

contacts = em.ContactMatrix(age_groups, np.identity(len(age_groups)))
matrices_contact = [contacts]

# Matrices contact
time_changes_contact = [1]
time_changes_region = np.arange(1, 94, 7).tolist()

### Set the parameters and initial conditions of the model and bundle everything together to simulate

In [3]:
# Instantiate model
model = em.PheSEIRModel()

# Initial number of susceptibles
susceptibles = [
    [68124, 299908, 773741, 668994, 1554740, 1632059, 660187, 578319],
    [117840, 488164, 1140597, 1033029, 3050671, 2050173, 586472, 495043],
    [116401, 508081, 1321675, 1319046, 2689334, 2765974, 1106091, 943363],
    [85845, 374034, 978659, 1005275, 2036049, 2128261, 857595, 707190],
    [81258, 348379, 894662, 871907, 1864807, 1905072, 750263, 624848],
    [95825, 424854, 1141632, 1044242, 2257437, 2424929, 946459, 844757],
    [53565, 237359, 641486, 635602, 1304264, 1499291, 668999, 584130]]
dI = 4
dL = 4

# Initial R number by region
psis = gamma.rvs(31.36, scale=1/224, size=len(regions))
initial_r = np.multiply(dI*psis, np.divide(np.square((dL/2)*psis+1), 1 - 1/np.square((dI/2)*psis+1)))

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, 94, 0.5).tolist()

# List of initial conditions and parameters that characterise the model
parameters = [
    1, susceptibles, np.ones((len(regions), len(age_groups))).tolist(), np.zeros((len(regions), len(age_groups))).tolist(),
    np.zeros((len(regions), len(age_groups))).tolist(), np.zeros((len(regions), len(age_groups))).tolist(), np.zeros((len(regions), len(age_groups))).tolist(),
    np.ones((len(regions), len(times))).tolist(), dL, dI, 0.5]

# Simulate using the ODE solver from scipy
scipy_method = 'RK45'

output_scipy_solver = model.simulate(
    parameters, times, matrices_contact, time_changes_contact,
    regions, initial_r, matrices_region, time_changes_region,
    method=scipy_method)

# Use different time steps for personalised solver
outputs_my_solver = []
time_steps = [0.5, 0.25, 0.05, 10**(-3)]

for ts in time_steps:
    # Update value of time step in parameters vector
    parameters[-1] = ts

    # Simulate using the 'homemade' discretised version of the ODE solver
    outputs_my_solver.append(model.simulate(
        parameters, times, matrices_contact, time_changes_contact,
        regions, initial_r, matrices_region, time_changes_region,
        method='my-solver'))

### Plot the comparments of the two methods against each other

In [4]:
# Group outputs together
outputs = outputs_my_solver
outputs.append(output_scipy_solver)

# Trace names - represent the solver used for the simulation
trace_name = ['my-solver with delta_t = {}'.format(ts) for ts in time_steps]
trace_name.append('scipy-solver {}'.format(scipy_method))

# Compartment list - type and age
comparments = []
for n in model.output_names():
    comparments.extend(['{} for ages {} in region {}'.format(n, a, regions[parameters[0]-1]) for a in age_groups])

for c, comparment in enumerate(comparments):
    # Plot (scatter plot for each comparment)
    fig = go.Figure()

    for o, out in enumerate(outputs):
        fig.add_trace(
            go.Scatter(
                y=out[:, c],
                x=times,
                mode='lines',
                name=trace_name[o]
            )
        )
    fig.update_layout(boxmode='group', title=comparment)
    fig.show()

### Number of new infections over all time for chosen region

In [5]:
model.new_infections(output=output_scipy_solver)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Log-likelihood of observed number of deaths 

In [6]:
model.new_infections(output=output_scipy_solver)

obs_death = [10, 12]
fatality_ratio = [0.1, 0.5]
time_to_death = [0.5] * len(times)

model.loglik_deaths(
    obs_death, output_scipy_solver, fatality_ratio,
    time_to_death, 0.5, 1)

ValueError: Wrong number of age groups for observed number                 of deaths.

### Log-likelihood of observed number of positive results 

In [7]:
obs_pos = [10, 12]
tests = [20, 30]
sens = 0.9
spec = 0.1

model.loglik_positive_tests(obs_pos, output_scipy_solver, tests, sens, spec, 10)

array([-11.95266477, -24.43528122])